<a href="https://colab.research.google.com/github/JAMES-YI/00_Tensorflow_Tutorials/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN implementation with basic Tensorflow operations

Codes from: https://pythonprogramming.net/rnn-tensorflow-python-machine-learning-tutorial/

Modified by JYI, 02/26/2020

In [6]:
'''
- hm_epochs
- chunk_size
- n_chunks
- rnn_size, number of nodes in LSTM 
- input first goes through a LSTM (128 units) -> FC layer (10 units) -> prediction of logits -> cost calculation
- output shape of RNN, (batch_size, num_classes)
'''

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

hm_epochs = 3 
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128


x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')
layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
          'biases':tf.Variable(tf.random_normal([n_classes]))}

x = tf.transpose(x, [1,0,2])
x = tf.reshape(x, [-1, chunk_size])
x = tf.split(x, n_chunks, 0)
print(len(x)) # decompose x as 28 tensors

lstm_cell = rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True,reuse=True)
outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction) )
optimizer = tf.train.AdamOptimizer().minimize(cost)


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    for epoch in range(hm_epochs):
        epoch_loss = 0
        for _ in range(int(mnist.train.num_examples/batch_size)):
            epoch_x, epoch_y = mnist.train.next_batch(batch_size)
            epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))

            _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
            epoch_loss += c

        print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
28


In [3]:
# def recurrent_neural_network(x):
#   '''
#   - x
#   '''
#   layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
#             'biases':tf.Variable(tf.random_normal([n_classes]))}

#   x = tf.transpose(x, [1,0,2])
#   x = tf.reshape(x, [-1, chunk_size])
#   x = tf.split(x, n_chunks, 0)
#   print(x)

#   lstm_cell = rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True,reuse=True)
#   outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

#   output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

#   return output

# def train_neural_network(x):
#     prediction = recurrent_neural_network(x)
#     cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction) )
#     optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    
#     with tf.Session() as sess:
#         sess.run(tf.initialize_all_variables())

#         for epoch in range(hm_epochs):
#             epoch_loss = 0
#             for _ in range(int(mnist.train.num_examples/batch_size)):
#                 epoch_x, epoch_y = mnist.train.next_batch(batch_size)
#                 epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))

#                 _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
#                 epoch_loss += c

#             print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

#         correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

#         accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
#         print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))

# train_neural_network(x)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 3 loss: 198.94611271470785
Epoch 1 completed out of 3 loss: 56.95425859466195
Epoch 2 completed out of 3 loss: 37.6426000604406
Accuracy: 0.9761
